### Análisis de Datos del Mercado de Criptomonedas con Binance API

- En la era digital actual, el mercado de criptomonedas ha emergido como un campo fascinante y dinámico, ofreciendo oportunidades únicas para el análisis de datos y la comprensión de las tendencias del mercado financiero. Nuestro proyecto se centra en explorar este sector vibrante utilizando una de las herramientas más poderosas en el mundo de las criptomonedas: la API de Binance.

- Binance, siendo una de las plataformas de intercambio de criptomonedas más grandes y reconocidas a nivel mundial, proporciona una API rica y robusta, permitiendo a los usuarios acceder a una amplia gama de datos del mercado en tiempo real. Esto incluye, pero no se limita a, datos históricos de precios, detalles de operaciones individuales, volúmenes de trading, y mucho más.

- El objetivo principal de este proyecto es aprovechar estos datos para obtener insights valiosos sobre el comportamiento del mercado de Bitcoin (BTC), la criptomoneda líder en el mercado. Nos centraremos en extraer datos históricos de precios y operaciones de BTC comparados con el USD, analizando las tendencias, volatilidades y posibles patrones de trading.

- Al final de este proyecto, esperamos tener un análisis detallado y multifacético del mercado de Bitcoin, brindando un panorama claro no solo para los entusiastas de las criptomonedas, sino también para aquellos interesados en las finanzas y la tecnología. Este desafío es una excelente oportunidad para sumergirnos en el mundo del análisis de datos financieros y abrir la puerta a futuras investigaciones y proyectos en este apasionante campo.

In [20]:
# paquetes necesarios
import requests
import pandas as pd
import hmac
import hashlib
import time
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
from decouple import config
from binance.client import Client
api_key = config('API_KEY')
api_secret = config('API_SECRET')
api_limit = 1000
requestDays = 3365
limit = 1000

## Descargar ultimos precios

In [21]:
# Crear una instancia del cliente de Binance
client = Client(api_key, api_secret)

# Calcular las fechas para el último mes
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=requestDays)

# Convertir las fechas a milisegundos (que es el formato que la API de Binance espera)
start_date_timestamp = int(start_date.timestamp() * 1000)
end_date_timestamp = int(end_date.timestamp() * 1000)

# Obtener los datos del precio del Bitcoin para el último mes con intervalo de 1 minuto
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE

In [22]:

# Inicializar una lista para almacenar los datos
all_klines = []

# Hacer llamadas sucesivas para obtener más datos (limitando cada llamada a 500)

while start_date_timestamp < end_date_timestamp:
    klines = client.get_klines(symbol=symbol, interval=interval, startTime=start_date_timestamp, endTime=end_date_timestamp, limit=limit)
    
    # Verificar si hay datos en la respuesta
    if not klines:
        break
    
    all_klines.extend(klines)
    
    # Actualizar el startTime para la siguiente llamada
    start_date_timestamp = int((pd.to_datetime(klines[-1][0], unit='ms') + timedelta(minutes=1)).timestamp() * 1000)

# Comprobar si se recuperaron datos antes de crear el DataFrame
if all_klines:
    # Crear un DataFrame de Pandas con todos los datos
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(all_klines, columns=columns)

    # Convertir el timestamp a formato de fecha y hora legible
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Mostrar el DataFrame
    df.info()
else:
    print("No se encontraron datos para el período especificado.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3432493 entries, 0 to 3432492
Data columns (total 12 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   timestamp                     datetime64[ns]
 1   open                          object        
 2   high                          object        
 3   low                           object        
 4   close                         object        
 5   volume                        object        
 6   close_time                    int64         
 7   quote_asset_volume            object        
 8   number_of_trades              int64         
 9   taker_buy_base_asset_volume   object        
 10  taker_buy_quote_asset_volume  object        
 11  ignore                        object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 314.3+ MB


In [24]:
df.sample(5)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
2473619,2022-05-06 21:31:00,36071.46000000,36073.64000000,36059.99000000,36062.55000000,12.40598000,1651872719999,447443.01116490,352,6.82608000,246176.07726750,0
2437764,2022-04-11 23:56:00,39497.11000000,39500.00000000,39491.27000000,39491.28000000,15.01883000,1649721419999,593204.13248390,413,7.79065000,307706.05905360,0
1550861,2020-08-03 03:28:00,11163.77000000,11177.74000000,11162.57000000,11176.60000000,29.15261800,1596425339999,325646.71524562,442,14.42326200,161117.02165262,0
2592890,2022-07-28 17:22:00,23968.50000000,23971.22000000,23940.49000000,23943.40000000,187.81254000,1659028979999,4499778.00432070,5382,81.63492000,1956003.88863590,0
127438,2017-11-13 22:57:00,6419.99000000,6419.99000000,6419.99000000,6419.99000000,0.00470000,1510613879999,30.17395300,1,0.00470000,30.17395300,0


In [25]:
## save df to backup.json
df.to_json('backup.json')